# get_LDA_df.ipynb

This notebook:
* Trains the LDA model on the podcast transcripts. 
    * Tokenizes the text using nltk stopwords. 
    * Uses sklearn's LDA CountVectorizer and LatentDirichletAllocation.
* Writes out the top 50 words for each topic into a csv. 
* Saves the LDA Document-Topic distributions as features into the df, and writes this df out to csv. 

In [1]:
import os
import json
import sys

import pandas as pd
import numpy as np

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import utils_general

# set random seed
np.random.seed(0)

# set pandas to show all the cols in this notebook
pd.set_option('display.max_columns', None)

# read in df
df = pd.read_csv("./csv/df.csv")

# if no words were transcribed, replace nan's with empty string
df["transcript"] = df["transcript"].fillna("")

# read in data samples from the df
data_samples = list(df["transcript"])

n_top_words = 50
            
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_words_idx = topic.argsort()[:-n_top_words - 1:-1]  # Get indices of top n words
        top_words = [feature_names[i] for i in top_words_idx]
        print("\n", topic_idx+1, ":", end=" ")
        for word in top_words:
            print(word, end=", ")
            
def save_top_words(model, feature_names, n_top_words):
    with open("./LDA_topics.csv", "w") as f:
        for topic_idx, topic in enumerate(model.components_):
            top_words_idx = topic.argsort()[:-n_top_words - 1:-1]  # Get indices of top n words
            top_words = [feature_names[i] for i in top_words_idx]
            f.write(f"{topic_idx+1},")
            for word in top_words:
                f.write(word+",")
            f.write("\n")
            
# nltk stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# tokenize and filter stopwords
tokenized_documents = [word_tokenize(doc.lower()) for doc in data_samples]
filtered_documents = [
    [word for word in doc if word.isalnum() and word not in stop_words]
    for doc in tokenized_documents
]

count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform([" ".join(doc) for doc in filtered_documents])

print("Fitting LDA model...")
n_topics = 100
lda = LatentDirichletAllocation(
    n_components=n_topics,
    learning_method="online",
    random_state=0, 
    max_iter=5,
    evaluate_every=1,
    verbose=1
    # we use the default value for perplexity tolerance
)
lda.fit(X)

count_feature_names = count_vectorizer.get_feature_names_out()
print_top_words(lda, count_feature_names, n_top_words)
save_top_words(lda, count_feature_names, n_top_words)

# Transform the fitted LDA model to get document-topic distribution
document_topic_probs = lda.transform(X)
print("Doc #0 probabilities =", document_topic_probs[0])
    
# Create a new DataFrame with the original data and add the topic probabilities
df = pd.concat([df, pd.DataFrame(document_topic_probs, columns=[f'Topic_{i+1}_Probability' for i in range(n_topics)])], axis=1)

# drop columns containing 'Unnamed' in their name because these are old indexes
columns_to_drop = [col for col in df.columns if "Unnamed" in col]
df = df.drop(columns=columns_to_drop)

# Display the new DataFrame
display(df)

csv_path = "./csv/df-LDA_topics.csv"
utils_general.delete_file_if_already_exists(csv_path)
df.to_csv(csv_path, header=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/grads/m/mariateleki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Fitting LDA model...
iteration: 1 of max_iter: 5, perplexity: 9715.0403
iteration: 2 of max_iter: 5, perplexity: 9687.5349
iteration: 3 of max_iter: 5, perplexity: 9673.9227
iteration: 4 of max_iter: 5, perplexity: 9667.2046
iteration: 5 of max_iter: 5, perplexity: 9662.9406

 1 : cartilage, hydrogen, o2, tentacles, philo, calcification, deployments, lipids, leona, violins, avascular, celiac, unifying, potts, agreeance, pythagoras, copd, brca, pancreatic, dissolution, whipple, calcified, seasonality, v3, emits, hardening, cerberus, reactors, broskis, cervantes, pth, thruster, scalding, perot, colic, underlies, radcast, fistula, popularize, malignancies, asin, metabolically, rpsb, frcr, malformations, tca, cholo, cardiopulmonary, musing, landmine, 
 2 : blah, language, english, say, word, words, use, learning, learn, speak, spanish, means, something, okay, example, different, languages, would, speaking, used, understand, native, let, lesson, another, read, one, right, reading, brazil, t

,show_uri,show_name,show_description,publisher,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,language,transcript,transcript_length,Topic_1_Probability,Topic_2_Probability,Topic_3_Probability,Topic_4_Probability,Topic_5_Probability,Topic_6_Probability,Topic_7_Probability,Topic_8_Probability,Topic_9_Probability,Topic_10_Probability,Topic_11_Probability,Topic_12_Probability,Topic_13_Probability,Topic_14_Probability,Topic_15_Probability,Topic_16_Probability,Topic_17_Probability,Topic_18_Probability,Topic_19_Probability,Topic_20_Probability,Topic_21_Probability,Topic_22_Probability,Topic_23_Probability,Topic_24_Probability,Topic_25_Probability,Topic_26_Probability,Topic_27_Probability,Topic_28_Probability,Topic_29_Probability,Topic_30_Probability,Topic_31_Probability,Topic_32_Probability,Topic_33_Probability,Topic_34_Probability,Topic_35_Probability,Topic_36_Probability,Topic_37_Probability,Topic_38_Probability,Topic_39_Probability,Topic_40_Probability,Topic_41_Probability,Topic_42_Probability,Topic_43_Probability,Topic_44_Probability,Topic_45_Probability,Topic_46_Probability,Topic_47_Probability,Topic_48_Probability,Topic_49_Probability,Topic_50_Probability,Topic_51_Probability,Topic_52_Probability,Topic_53_Probability,Topic_54_Probability,Topic_55_Probability,Topic_56_Probability,Topic_57_Probability,Topic_58_Probability,Topic_59_Probability,Topic_60_Probability,Topic_61_Probability,Topic_62_Probability,Topic_63_Probability,Topic_64_Probability,Topic_65_Probability,Topic_66_Probability,Topic_67_Probability,Topic_68_Probability,Topic_69_Probability,Topic_70_Probability,Topic_71_Probability,Topic_72_Probability,Topic_73_Probability,Topic_74_Probability,Topic_75_Probability,Topic_76_Probability,Topic_77_Probability,Topic_78_Probability,Topic_79_Probability,Topic_80_Probability,Topic_81_Probability,Topic_82_Probability,Topic_83_Probability,Topic_84_Probability,Topic_85_Probability,Topic_86_Probability,Topic_87_Probability,Topic_88_Probability,Topic_89_Probability,Topic_90_Probability,Topic_91_Probability,Topic_92_Probability,Topic_93_Probability,Topic_94_Probability,Topic_95_Probability,Topic_96_Probability,Topic_97_Probability,Topic_98_Probability,Topic_99_Probability,Topic_100_Probability
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj,en,"Hello, hello, hello everyone. This is Katie an...",1716,0.000012,0.000012,0.002161,0.002365,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.034438,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.001214,0.000012,0.000012,0.003613,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.001192,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.030293,0.000012,0.000012,0.000012,0.000012,0.008942,0.263176,0.000012,0.360638,0.000012,0.000012,0.004425,0.000012,0.000012,0.007946,0.000012,0.000012,0.225073,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.001251,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.052268
1,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa,en,Welcome to Insi